In [17]:
import os
import sys
sys.path.append(os.path.join(os.getcwd().replace("notebooks/distilled_models", "")))

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from config import Config
from data.dataset import NetworkDataset, load_datasets
from model_config import MLP_Models
from compact.knowledge_distillation import KnowledgeDistillation

In [18]:
conf = Config()
load_models = MLP_Models()
teacher_model = load_models.mlp_4
teacher_model.load()
student_model = load_models.light_mlp_4
kd = KnowledgeDistillation(
    teacher=teacher_model,
    student=student_model,
    device=conf.device,
    distillation="feature"
)

Checkpoint loaded from /global/D1/homes/jorgetf/Network-Packet-ML-Model/checkpoint/large_model/mlp_4.pth!


In [19]:
X_train, y_train, X_val, y_val, X_test, y_test = load_datasets(conf.datasets, "mlp")

# create train, val and test dataloaders
train_dataset = NetworkDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, conf.batch_size, shuffle=True)

val_dataset = NetworkDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, conf.batch_size, shuffle=True)

test_dataset = NetworkDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, conf.batch_size)

In [20]:
data, labels = next(iter(train_loader))
print(data.shape, labels.shape)

torch.Size([32, 513]) torch.Size([32])


In [21]:
acc, train_loss, val_loss = kd.train_kd(train_loader, val_loader, conf.epochs)

Epoch: 1/10, Accuracy: 72.63%, Train loss: 11515435.9197, Val loss: 0.6649
Epoch: 2/10, Accuracy: 75.72%, Train loss: 4870293.6954, Val loss: 0.5705
Epoch: 3/10, Accuracy: 79.16%, Train loss: 4833730.4675, Val loss: 0.4566
Epoch: 4/10, Accuracy: 80.09%, Train loss: 4936344.6789, Val loss: 0.4372
Epoch: 5/10, Accuracy: 81.44%, Train loss: 4799268.6308, Val loss: 0.4130
Epoch: 6/10, Accuracy: 82.32%, Train loss: 4407202.7046, Val loss: 0.3662
Epoch: 7/10, Accuracy: 80.54%, Train loss: 4490983.5511, Val loss: 0.4445
Epoch: 8/10, Accuracy: 62.02%, Train loss: 4121656.9784, Val loss: 0.6326
Epoch: 9/10, Accuracy: 81.67%, Train loss: 4616731.1706, Val loss: 0.4102
Epoch: 10/10, Accuracy: 81.98%, Train loss: 4513308.9914, Val loss: 0.3956


In [22]:
# test teacher accuracy
loss, accuracy = teacher_model.evaluate(test_loader)
print(f"Teacher accuracy: {100*accuracy:.2f}%")

Teacher accuracy: 91.51%


In [23]:
# test student accuracy
loss, accuracy = student_model.evaluate(test_loader)
print(f"Student accuracy: {100*accuracy:.2f}%")

Student accuracy: 81.78%


In [24]:
student_model.save()

Checkpoint saved at /global/D1/homes/jorgetf/Network-Packet-ML-Model/checkpoint/compressed_model/light_mlp_4.pth
